# Crack identification procedure

In [ ]:
%matplotlib widget
import ibvpy.api as ib
import matplotlib.pylab as plt
from matplotlib import cm
from bmcs_shear.dic_crack import\
    DICGridTri, DICStateFields, DICCrackList, \
    DICAlignedGrid, DICGrid
import numpy as np
np.seterr(divide ='ignore', invalid='ignore');
import collections
collections.Iterable = collections.abc.Iterable

In [ ]:
#dic_grid = DICGridTri(U_factor=100, dir_name='B1_TV1', t=1, d_x=22, d_y=22, n_T=10)
dic_grid = DICGrid(U_factor=100, dir_name='B1_TV1', t=1)

In [ ]:
dic_grid.read_beam_design()

In [ ]:
from bmcs_shear.dic_crack import DICStressProfile
dsf = DICStateFields(dic_grid=dic_grid, R=7, omega_threshold=0.15, verbose_eval=True)
dsf.tmodel_.trait_set(epsilon_f=0.00025, epsilon_0 = 0.000059)

# l_cb = dic_grid.d_x * 1
# dsf.tmodel_.trait_set(E=5000, c_T=0, nu=0.18, epsilon_0=0.0005, epsilon_f=0.02/l_cb, eps_max=0.005);
dsf.tmodel_.interact()

In [ ]:
dsf.eval()
dcl = DICCrackList(dsf=dsf, corridor_left=3, corridor_right=1)

In [ ]:
dcl.identify_cracks()

In [ ]:
dcl.interact()

In [ ]:
dcl.dsf.X_fe_KLa[(0,0,-1,-1),(0,0,-1,-1),(0,1,0,1)]

In [ ]:
dcl.dsf.dic_grid.X_IJa[(0,0,-1,-1),(0,0,-1,-1),(0,1,0,1)]

In [ ]:
dcl.dsf.X_ipl_MNa[(0,0,-1,-1),(0,0,-1,-1),(0,1,0,1)]

In [ ]:
dcl.items['0'].x_t_crc_Ka

In [ ]:
dcl.items['1'].x_t_crc_Ka

In [ ]:
for i,v in dcl.items.items():
    print(i, len(v.x_t_crc_Ka))

In [ ]:
%%capture

from matplotlib import animation, rc
from IPython.display import HTML
from traitlets import HasTraits

fig = plt.figure()
fig.set_size_inches(12, 6)

In [ ]:
class CrackListAnimator(HasTraits):
    
    
    def init(self):
        print('init')
        self.t_dic_T = np.hstack([dcl.dsf.dic_grid.t_dic_T, np.ones(10)])        
    
    def plot(self, i):
        fig.clear()
        t = self.t_dic_T[i]
        axes = dcl.subplots(fig)
        dcl.dsf.dic_grid.t = t
        dcl.update_plot(axes)
        
    def mp4_video(self):
        n_t = len(self.t_dic_T)
        # call the animator. blit=True means only re-draw the parts that have changed.
        anim = animation.FuncAnimation(fig, self.plot, init_func=self.init,
                                       frames=n_t, interval=300, blit=True)
        return anim.save("cracking_animation.gif")
    
    def html5_video(self):
        n_t = len(self.t_dic_T)
        # call the animator. blit=True means only re-draw the parts that have changed.
        anim = animation.FuncAnimation(fig, self.plot, init_func=self.init,
                                       frames=n_t, interval=300, blit=True)
        return anim.to_html5_video()

In [ ]:
anim = CrackListAnimator()
anim.init()
anim.mp4_video()

In [ ]:
anim = CrackListAnimator()
anim.init()
with open('cracking_animation.html','w') as html_video_file:
    html_video_file.write(anim.html5_video())

In [ ]:
html_video_file = open('cracking_animation.html','r')
HTML(html_video_file.read())